# Geospatial BI & Data Viz Companies - Part 2

In [72]:
# import pymongo to connect Python with MongoDB + GEOSPHERE for geoquery
from pymongo import MongoClient, GEOSPHERE
# to work with stats
import pandas as pd
# to work with dataframes
import numpy as np
# to work with json
import json
from pandas.io.json import json_normalize
from flatten_json import flatten
# to plot
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

In [75]:
# connecting on default host and port
client = MongoClient ('localhost', 27017)

# loading the database
db = client['companies_segmentation']

# getting the collection
companies_segmentation = db['companies_segmentation']

In [74]:
# creating index for geoqueries -->  directly in MongoDB
#csv 'all_offices_segmentation.csv'

In [76]:
# defining the 'near' geoquery function to get the closest offices with the parameters
# max distance for search_area is in meters

def geoquery_per_geopoint(longitude, latitude, distance): 

    df_aux = pd.DataFrame(companies_segmentation.find({"geoquery":{"$near":{"$geometry": 
                    {"type":"Point",
                    "coordinates": [longitude , latitude]},
                    "$maxDistance": distance}}}),
                    {'name':1,'type':1,'number_of_employees_bins':1,'founded_year_bins':1,'segment':1})
    return df_aux

In [77]:
total_data = pd.DataFrame(companies_segmentation.find())

In [78]:
i = 0
print(total_data.longitude[i], total_data.latitude[i])

-122.4169244 37.7768052


In [79]:
print(total_data['type'][i])

big company


In [81]:
i=0
distance=5000
longtotal_data.loc[i, "longitude"], total_data.loc[i,"latitude"]
one_geoquery = geoquery_per_geopoint(longitude, latitude, distance)

NameError: name 'longitude' is not defined

In [ ]:
# # analyze the results of each query and create an object/dict so we can append it to the final dataframe of results

# def results_per_geoquery(df_aux):
    
#     companies = len(df_aux)
#     startups,bigcompanies,old,recent,gaming,tech = 0,0,0,0,0,0
#     for i in range(len(df_aux):
#         startups +=1 if (df_aux['type'] == 'startup')
#         bigcompanies +=1 if (df_aux['type'][i] == 'big company')
#         old +=1 if df_aux['founded_year_bins'][i] == 'q1')
#         recent +=1 if (df_aux['founded_year_bins'][i] == 'q4')
#         gaming +=1 if (df_aux['segment'][i] == 'gaming')
#         tech +=1 if (df_aux['segment'][i] == 'tech')
#     results_query = {'companies':companies,
#                      'startups':startups*100/companies,
#                      'bigcompanies':bigcompanies*100/companies,
#                      'old':old*100/companies,
#                      'recent':recent*100/companies,
#                      'gaming':gaming*100/companies,
#                      'tech':tech*100/companies}              
    
#     return results_query

In [80]:
# analyze the results of each query and create an object/dict so we can append it to the final dataframe of results

def results_per_geoquery(df_aux):
    
    companies = len(df_aux)
    startups = len(df_aux[df_aux['type'] == 'startup'])*100/companies
    bigcompanies = len(df_aux[df_aux['type'] == 'big company'])*100/companies
    old = len(df_aux[df_aux['founded_year_bins'] == 'q1'])*100/companies
    recent = len(df_aux[df_aux['founded_year_bins'] == 'q4'])*100/companies
    gaming = len(df_aux[df_aux['segment'] == 'gaming'])*100/companies
    tech = len(df_aux[df_aux['segment'] == 'tech'])*100/companies
    results_query = {'companies':companies,'startups':startups,'bigcompanies':bigcompanies,
                     'old':old,'recent':recent,'gaming':gaming,'tech':tech}              
    
    return results_query

In [63]:
def concat_results_df(results_df, results_query):
    # add query as a new row of the total dataset
    #results_df.append(results_query,ignore_index=True)
    results_df = pd.concat(results_df,results_query,axis=0,ignore_index=True)
    return results_df

In [66]:
def loop_geoquery_df(df,distance):
    # loop to apply our geoquery to each given location of our database
    for i in range(len(df)):
        longitude, latitude = df.loc[i, "longitude"], df.loc[i,"latitude"]
        results_query = results_per_geoquery(geoquery_per_geopoint(longitude, latitude, distance))
        results_df = concat_results_df(results_df, results_query)
    results_total_df = pd.concat([results_df,df], axis=1,ignore_index=True)
    return results_total_df

In [71]:
# create empty total DataFrame to join results dataframe to our initial offices dataframe

# create empty total results dataframe to append all subdataframes (results) of each query
results_df = pd.DataFrame(columns=['companies','startups','big_companies','recent','old','gaming','tech'])

results_total_df = loop_geoquery_df(total_data,5000)
results_total_df.head()

ValueError: Shape of passed values is (271, 15), indices imply (5, 15)

In [ ]:
len(total_data[total_data['type'] == 'startup'])*100/len(total_data)

In [ ]:
d= {'1':'a', '2':'b', '3':'c'}
d
j = pd.DataFrame(columns=['1','2','3'])
#concatenate(j,d)
f = j.append(d,ignore_index=True)
f

In [ ]:
near_geoquery(total_data, 5000)

In [ ]:
i = 0
print(total_data.longitude[i], total_data.latitude[i])

In [ ]:
print(total_data.loc[i, "longitude"] , total_data.loc[i,"latitude"])

In [70]:
i = 0
search_area = 5000 # meters = 5 km

cursor = list(companies_segmentation.find({"geoquery":{"$near":{"$geometry": 
                {"type":"Point",
                "coordinates": [total_data.longitude[i] , total_data.latitude[i]]},
                "$maxDistance": search_area}}}))

In [ ]:
# for geo in range(len(geoquery_array)):
#     # we define the limit of our search area
#     search_area=10000
#     # we stock cursor results in a dataframe
#     appended_data = []
#     data_geo = pd.DataFrame(list(near_geoquery(geo['longitude'], geo['latitude'], search_area)))
#     # store DataFrame in list
#     appended_data.append(data_geo)
#     # we concat dataframes
#     appended_data = pd.concat(appended_data, axis=1)

# appended_data.head()

In [ ]:
# defining the 'near' geoquery function to get the closest offices with the parameters
# max distance for search_area is in meters

# def near_geoquery(df,search_area):
#     # we will create different lists for the given results:
#     total_companies = [] 
#     startups = []                 #--> we want a good ratio over total companies
#     big_companies = []             #--> we want a good ratio 
#     recent = []  #q4 founded year --> we want to be the closest possible
#     old = []     #q1 founded year --> we prefer to be the most far away from them
#     gaming = []                  #--> we want to cover our team interests / networking
#     tech = []                    #--> we want software engineers near us/ours
    
#     # loop to apply our geoquery to each given location of our database
#     for i in range(len(df)):
#         print(i)
#         df_aux = pd.DataFrame(companies_segmentation.find({"geopoint":{"$near":{"$geometry": 
#                 {"type":"Point",
#                 "coordinates": [df.loc[i, "longitude"] , df.loc[i,"latitude"]]},
#                 "$maxDistance": search_area}}})
                  
#         print('companies:',len(df_aux))
#         print('startups',len(df_aux[df_aux.type == 'startup'])/companies)
#         print('big_companies',len(df_aux[df_aux.type == 'big company'])/companies)
#         print('recent',len(df_aux[df_aux.founded_year_bins == 'q1'])/companies)
#         print('old',len(df_aux[df_aux.founded_year_bins == 'q4'])/companies)
#         print('gaming',len(df_aux[df_aux.segment == 'gaming'])/companies) 
#         print('tech',len(df_aux[df_aux.segment == 'tech'])/companies)

#         total_companies.append(len(df_aux))
# #         # we will append each geoquery results to our lists   
# #         companies = len(df_aux)
# #         # but for all criteria, we will normalize, so it won't depend on the nb of companies per city/country 
# #         total_companies.append(companies)
# #         startups.append(len(df_aux[df_aux.type == 'startup'])/companies)   
# #         big_companies.append(len(df_aux[df_aux.type == 'big company'])/companies)        
# #         recent.append(len(df_aux[df_aux.founded_year_bins == 'q1'])/companies) 
# #         old.append(len(df_aux[df_aux.founded_year_bins == 'q4'])/companies) 
# #         gaming.append(len(df_aux[df_aux.segment == 'gaming'])/companies) 
# #         tech.append(len(df_aux[df_aux.segment == 'tech'])/companies) 
    
# #     # the function will return a dataframe with specific results (criteria) from cursors geoqueries:
# #     df['total_companies'] = total_companies    
# #     df['r_startups'] = startups
# #     df['r_big_companies'] = big_companies
# #     df['r_recent'] = recent
# #     df['r_old'] = old
# #     df['r_gaming'] = gaming
# #     df['r_tech'] = tech

#     return total_companies